<font size="+3"><strong>Wrangling Data with MongoDB </strong></font>

In [ ]:
from pprint import PrettyPrinter
import pandas as pd
from pymongo import MongoClient

# Prepare Data

In [ ]:
client = MongoClient(host="localhost", port=27017)

In [ ]:
#list(client.list_databases())
pp.pprint(list(client.list_databases()))
#getsizeof()

In [2]:
db = client["air-quality"]

In [ ]:
#db.list_collections
for c in db.list_collections():
    print(c["name"])

In [ ]:
nairobi = db["nairobi"]

In [ ]:
nairobi.count_documents({})

In [ ]:
result = nairobi.find_one({})
pp.pprint(result)

In [ ]:
nairobi.distinct("metadata.site")

In [ ]:
#nairobi.count_documents({"metadata.site": 6})
print("Documents from site 6:", nairobi.count_documents({"metadata.site": 6}))
print("Documents from site 29:", nairobi.count_documents({"metadata.site": 29}))

In [ ]:

result = nairobi.aggregate(
    [
        {"$group": {"_id": "$metadata.site", "count": {"$count": {} }}} 
    ]
)

pp.pprint(list(result))

In [ ]:
nairobi.distinct("metadata.measurement")

In [ ]:
result = nairobi.find({"metadata.measurement": "P2"}).limit(3)
pp.pprint(list(result))

In [ ]:
result = nairobi.aggregate(
    [
         {"$match": {"metadata.site": 6}},
        {"$group": {"_id": "$metadata.measurement", "count": {"$count": {} }}} 
    ]
)

pp.pprint(list(result))

In [ ]:
result = nairobi.aggregate(
    [
         {"$match": {"metadata.site": 29}},
        {"$group": {"_id": "$metadata.measurement", "count": {"$count": {} }}} 
    ]
)
pp.pprint(list(result))

In [ ]:
result = nairobi.find(
    {"metadata.site":29, "metadata.measurement":"P2"}
  
)
pp.pprint(result.next())

In [ ]:
result = nairobi.find(
    {"metadata.site":29, "metadata.measurement":"P2"},
    projection = {"P2": 1, "timestamp": 1, "_id": 0}
)
#pp.pprint(result.next())

In [ ]:
df = pd.DataFrame(result).set_index("timestamp")
df.head()

In [ ]:
# Check your work
assert df.shape[1] == 1, f"`df` should have only one column, not {df.shape[1]}."
assert df.columns == [
    "P2"
], f"The single column in `df` should be `'P2'`, not {df.columns[0]}."
assert isinstance(df.index, pd.DatetimeIndex), "`df` should have a `DatetimeIndex`."

In [ ]:
#df.to_csv("task3/P2-nairobi-1.csv")